In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import xarray as xr

In [ ]:
try:
    from bapsf_motion.motion_builder.exclusions import CircularExclusion, DividerExclusion
except ModuleNotFoundError:
    from pathlib import Path

    HERE = Path().cwd()
    BAPSF_MOTION = (HERE / ".." / ".." / ".." / "src").resolve()
    sys.path.append(str(BAPSF_MOTION))
    
    from bapsf_motion.motion_builder.exclusions import CircularExclusion, DividerExclusion
    

In [ ]:
plt.rcParams.update(
    {
        # "figure.figsize": [12, 0.56 * 12],
        "figure.figsize": [10, 0.8 * 10],
        "font.size": 16,
    }
)

## Create LaPD XY exclusion mask using elementry exclusions

Create the seeding boolean mask.  In the boolean mask a `True` value indicates a point allowed for a motion list and a `False` value indicaes an exclusion zone.

In [ ]:
size = 221
side = np.linspace(-55, 55, num=size)
ds = xr.Dataset(
    {"mask": (("x", "y"), np.ones((size, size), dtype=bool))},
    coords={
        "x": side,
        "y": side,
    },
)

ds.mask.plot(x="x", y="y");

Add a circular XY exclusion that simulates the LaPD inner diameter

In [ ]:
ex1 = CircularExclusion(ds, radius=50)
ds["mask"].plot(x="x", y="y");

Let's add boundaries representing the exclusion cone from the East port.

In [ ]:
x_pivot = 58.771
cone_half_angle = 40  # degrees

intercept = np.abs(x_pivot) * np.tan(np.radians(cone_half_angle))
slope = intercept / np.abs(x_pivot)

print(isinstance(intercept, float))

ex2 = DividerExclusion(
    ds,
    mb=(-slope, intercept),
    exclude="+e1",
)
ex3 = DividerExclusion(
    ds,
    mb=(slope, -intercept),
    exclude="-e1",
)
ds["mask"].plot(x="x", y="y");

In [ ]:
ds

## Generate a LaPD mask using LaPDXYExclusion

In [ ]:
from bapsf_motion.motion_builder.exclusions import LaPDXYExclusion

In [ ]:
size = 100
side = np.linspace(-55, 55, num=size)
ds = xr.Dataset(
    {"mask": (("x", "y"), np.ones((size, size), dtype=bool))},
    coords={
        "x": side,
        "y": side,
    },
)

ds.mask.plot(x="x", y="y");

The default settings for the for `LaPDXYExclusion` are the same as the manually constructed exclusion mask above.

In [ ]:
ex = LaPDXYExclusion(ds)

ds.mask.plot(x="x", y="y")

mb = ex.composed_exclusions[1].mb
plt.plot(side, mb[0] * side + mb[1], color="blue")

mb = ex.composed_exclusions[2].mb
plt.plot(side, mb[0] * side + mb[1], color="red");

Additionally, port locations can be specified using values of `E`, `East`, `W`, `West`, `T`, `Top`, `B`, and `Bottom` (all are case insensitive).

In [ ]:
ds = xr.Dataset(
    {"mask": (("x", "y"), np.ones((size, size), dtype=bool))},
    coords={
        "x": side,
        "y": side,
    },
)

ex = LaPDXYExclusion(ds, port_location="T")

ds.mask.plot(x="x", y="y")

mb = ex.composed_exclusions[1].mb
plt.plot(side, mb[0] * side + mb[1], color="blue")

mb = ex.composed_exclusions[2].mb
plt.plot(side, mb[0] * side + mb[1], color="red");

For non-standard ports an angle (in degrees) can be given instead.

In [ ]:
ds = xr.Dataset(
    {"mask": (("x", "y"), np.ones((size, size), dtype=bool))},
    coords={
        "x": side,
        "y": side,
    },
)

ex = LaPDXYExclusion(ds, port_location=135)

ds.mask.plot(x="x", y="y")

mb = ex.composed_exclusions[1].mb
plt.plot(side, mb[0] * side + mb[1], color="blue")

mb = ex.composed_exclusions[2].mb
plt.plot(side, mb[0] * side + mb[1], color="red");

`LaPDXYExclusion` is a compound exclusiong, that is it is constructed from other base exclusions.  In this case, it build used one instance of the `CircularExclusion` and two instances of the `DividerExclusion`.

In [ ]:
ex.composed_exclusions

Just like base exclusesion, the `LaPDXYExclusion` configuration can be accesed via the `config` attribute.

In [ ]:
ex.config
